# Simple Test between NumPy and Numba

$$
\Gamma = \sqrt{\frac{\eta_H}{\eta_V} \kappa^2 + \eta_H \zeta_H}
$$

In [1]:
import numba
import cython
import numexpr
import numpy as np

%load_ext cython

In [ ]:
# Used cores by numba can be shown with (xy default all cores are used):
#print(numba.config.NUMBA_DEFAULT_NUM_THREADS)

# This can be changed with the following line
#numba.config.NUMBA_NUM_THREADS = 4

In [2]:
from empymod import filters
from scipy.constants import mu_0       # Magn. permeability of free space [H/m]
from scipy.constants import epsilon_0  # Elec. permittivity of free space [F/m]

res = np.array([2e14, 0.3, 1, 50, 1])             # nlay
freq = np.arange(1, 201)/20.                    # nfre
off = np.arange(1, 101)*1000                      # noff
lambd = filters.key_201_2009().base/off[:, None]  # nwav

aniso = np.array([1, 1, 1.5, 2, 1])
epermH = np.array([1, 80, 9, 20, 1])
epermV = np.array([1, 40, 9, 10, 1])
mpermH = np.array([1, 1, 3, 5, 1])

etaH = 1/res + np.outer(2j*np.pi*freq, epermH*epsilon_0)
etaV = 1/(res*aniso*aniso) + np.outer(2j*np.pi*freq, epermV*epsilon_0)
zetaH = np.outer(2j*np.pi*freq, mpermH*mu_0)

## NumPy

Numpy version to check result and compare times

In [3]:
def test_numpy(eH, eV, zH, l):
    return np.sqrt((eH/eV) * (l*l) + (zH*eH))

## Numba @vectorize

This is exactly the same function as with NumPy, just added the @vectorize decorater.

In [4]:
@numba.vectorize('c16(c16, c16, c16, f8)')
def test_numba_vnp(eH, eV, zH, l):
    return np.sqrt((eH/eV) * (l*l) + (zH*eH))

@numba.vectorize('c16(c16, c16, c16, f8)', target='parallel')
def test_numba_v(eH, eV, zH, l):
    return np.sqrt((eH/eV) * (l*l) + (zH*eH))

## Numba @njit

In [5]:
@numba.njit
def test_numba_nnp(eH, eV, zH, l):
    o1, o3 = eH.shape 
    o2, o4 = l.shape 
    out = np.empty((o1, o2, o3, o4), dtype=numba.complex128)
    for nf in numba.prange(o1):
        for nl in numba.prange(o3):
            ieH = eH[nf, nl]
            ieV = eV[nf, nl]
            izH = zH[nf, nl]
            for no in numba.prange(o2):
                for ni in numba.prange(o4):
                    il = l[no, ni]
                    out[nf, no, nl, ni] = np.sqrt(ieH/ieV * il*il + izH*ieH)
    return out
                    
@numba.njit(nogil=True, parallel=True)
def test_numba_n(eH, eV, zH, l):
    o1, o3 = eH.shape 
    o2, o4 = l.shape 
    out = np.empty((o1, o2, o3, o4), dtype=numba.complex128)
    for nf in numba.prange(o1):
        for nl in numba.prange(o3):
            ieH = eH[nf, nl]
            ieV = eV[nf, nl]
            izH = zH[nf, nl]
            for no in numba.prange(o2):
                for ni in numba.prange(o4):
                    il = l[no, ni]
                    out[nf, no, nl, ni] = np.sqrt(ieH/ieV * il*il + izH*ieH)
    return out

## Run comparison for a small and a big matrix

In [6]:
eH = etaH[:, None, :, None]
eV = etaV[:, None, :, None]
zH = zetaH[:, None, :, None]
l = lambd[None, :, None, :]

# Output shape
out_shape = (freq.size, off.size, res.size, filters.key_201_2009().base.size)

print(' Shape Test Matrix    ::', out_shape, '; total # elements:: '+str(freq.size*off.size*res.size*filters.key_201_2009().base.size))
print('------------------------------------------------------------------------------------------')

print(' NumPy                ::  ', end='')
# Get NumPy result for comparison
numpy_result = test_numpy(eH, eV, zH, l)
# Get runtime
%timeit test_numpy(eH, eV, zH, l)

print(' Numba @vectorize     ::  ', end='')
# Ensure it agrees with NumPy
numba_vnp_result = test_numba_vnp(eH, eV, zH, l)
if not np.allclose(numpy_result, numba_vnp_result, atol=0, rtol=1e-10):
    print(' * FAIL, DOES NOT AGREE WITH NumPy RESULT!')
# Get runtime
%timeit test_numba_vnp(eH, eV, zH, l)

print(' Numba @vectorize par ::  ', end='')
# Ensure it agrees with NumPy
numba_v_result = test_numba_v(eH, eV, zH, l)
if not np.allclose(numpy_result, numba_v_result, atol=0, rtol=1e-10):
    print(' * FAIL, DOES NOT AGREE WITH NumPy RESULT!')
# Get runtime
%timeit test_numba_v(eH, eV, zH, l)

print(' Numba @njit          ::  ', end='')
# Ensure it agrees with NumPy
numba_nnp_result = test_numba_nnp(etaH, etaV, zetaH, lambd)
if not np.allclose(numpy_result, numba_nnp_result, atol=0, rtol=1e-10):
    print(' * FAIL, DOES NOT AGREE WITH NumPy RESULT!')
# Get runtime
%timeit test_numba_nnp(etaH, etaV, zetaH, lambd)

print(' Numba @njit      par ::  ', end='')
# Ensure it agrees with NumPy
numba_n_result = test_numba_n(etaH, etaV, zetaH, lambd)
if not np.allclose(numpy_result, numba_n_result, atol=0, rtol=1e-10):
    print(' * FAIL, DOES NOT AGREE WITH NumPy RESULT!')
# Get runtime
%timeit test_numba_n(etaH, etaV, zetaH, lambd)

 Shape Test Matrix    :: (200, 100, 5, 201) ; total # elements:: 20100000
------------------------------------------------------------------------------------------
 NumPy                ::  1.02 s ± 27.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
 Numba @vectorize     ::  966 ms ± 12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
 Numba @vectorize par ::  781 ms ± 12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
 Numba @njit          ::  688 ms ± 12.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
 Numba @njit      par ::  368 ms ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
from empymod import versions
versions('HTML', add_pckg=[cython, numba], ncol=5)